# This program intend to do a point plot at Singapore map with the toggle function on flat_type legend. 
# However due to it take too long to run and it might confusing when there is too many data to plot. The program only select '2024-10' records.

In [ ]:
# Singapore Map visualization with legend and toggle functionality
#!pip install geopy
#!pip install folium
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from folium.plugins import MarkerCluster

url = "ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv" # read input file
df = pd.read_csv(url)
df =  df[df['month'] == '2024-10']

# Initialize the geolocator

geolocator = Nominatim(user_agent="geoapiExercises")

# Function to get latitude and longitude for an address
def get_lat_lon(row):
    try:
        location = geolocator.geocode(f"{row['block']} {row['street_name']}, Singapore")
        if location:
            return pd.Series([location.latitude, location.longitude])
        else:
            return pd.Series([None, None])
    except Exception as e:
        return pd.Series([None, None])

# Apply the geocoding function to get coordinates
df[['latitude', 'longitude']] = df.apply(get_lat_lon, axis=1)

# Drop rows with missing coordinates
dfn = df.dropna(subset=['latitude', 'longitude'])

# Create a folium map centered on Singapore
singapore_map = folium.Map(location=[1.3521, 103.8198], zoom_start=12)

# Color mapping for flat types
color_dict = {
    '1 ROOM': 'yellow',
    '2 ROOM': 'orange',
    '3 ROOM': 'blue',
    '4 ROOM': 'green',
    '5 ROOM': 'red',
    'EXECUTIVE': 'purple',
    'MULTI-GENERATION': 'purple'
}

# Create FeatureGroups for each flat type
feature_groups = {}
for flat_type in color_dict.keys():
    feature_groups[flat_type] = folium.FeatureGroup(name=flat_type)

# Add points to the respective FeatureGroup
for _, row in dfn.iterrows():
    flat_type = row['flat_type']
    if flat_type in feature_groups:
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=f"Flat Type: {flat_type}",
            icon=folium.Icon(color=color_dict.get(flat_type, 'gray'))
        ).add_to(feature_groups[flat_type])

# Add FeatureGroups to the map
for fg in feature_groups.values():
    fg.add_to(singapore_map)

# Add a LayerControl to toggle layers
folium.LayerControl(collapsed=False).add_to(singapore_map)

# Save the map with the legend and toggle functionality
singapore_map.save('singapore_map_with_toggle.html')